In [428]:
#Import the libraries I'll use.
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import re

In [480]:
#Starting by looking at a specific game page and looking what class is the box info.
r = requests.get("https://en.wikipedia.org/wiki/Donkey_Kong_(video_game)")
soups = bs(r.content,"lxml")

info_box = soups.find(class_="infobox hproduct")
info_rows = info_box.select("tr")
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above fn" colspan="2" style="font-size:125%;font-style:italic;">
  Donkey Kong
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Donkey_Kong_flier.jpg" title="Donkey Kong's 1981 North American arcade flier">
   <img alt="Donkey Kong's 1981 North American arcade flier" data-file-height="323" data-file-width="250" decoding="async" height="284" src="//upload.wikimedia.org/wikipedia/en/thumb/1/14/Donkey_Kong_flier.jpg/220px-Donkey_Kong_flier.jpg" srcset="//upload.wikimedia.org/wikipedia/en/1/14/Donkey_Kong_flier.jpg 1.5x" width="220"/>
  </a>
  <div class="infobox-caption" style="font-size: 95%;">
   1981 North American arcade flier
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space:nowrap;padding-right:0.65em;">
  <a href="/wiki/Video_game_developer" title="Video game developer">
   Developer(s)
  </a>
 </th>
 <td class="infobox-data">
  <div class="plainlist">
   <ul>
    <li>
     <a

In [488]:
info_box.find(class_="mw-collapsible-text")

In [466]:
#Creating a function to check specific conditions in another function.
def fixing_elems(elem):
    if elem.find("li",{"class":"mw-collapsible-text"}):
        pass
    elif elem.find("li"):# and elem!=elem.find(class_="mw-collapsible-content hlist"):
        return [li.get_text().replace("\xa0","").replace("\n","").strip() for li in elem.find_all("li")]
    elif elem.find("br"):
        elem = str(elem)
        elem = elem.replace("<br/>" , " ").replace("\n","").strip()
        elem = bs(elem)
        elem = elem.get_text().replace("\xa0","").replace("\n","").strip()
        return elem
    elif elem.find("sup"):
        elem = str(elem)
        elem = elem.replace("sup" , " ").replace("\n","").strip()
        elem = bs(elem)
        elem = elem.get_text().replace("\xa0","").replace("\n","").strip()
        return elem
    else:
        return elem.get_text().replace("\xa0","").replace("\n","").strip()

#Creating a function that takes a URL & and getting the data by each category to a dictionary.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index==0:
            games_info["Title"] = row.find("th").get_text(" ",strip=True)
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = fixing_elems(row.find("td"))
            games_info[key] = value
    
    return games_info

In [467]:
#Loaded the page I'm going to get all the video games list and iterate on each game to get the data
#with the help of the two functions i created above.
r = requests.get("https://en.wikipedia.org/wiki/List_of_video_games_considered_the_best")
soup_games = bs(r.content,"lxml")
games = soup_games.select(".wikitable.sortable i a")

wiki_path = "https://en.wikipedia.org"

game_info_list = []
for index, game in enumerate(games):
    try:
        relative_path = game["href"]
        full_wiki_path = wiki_path+relative_path
        fixing_elems(soup_games)
        title = game["title"]
        game_info_list.append(get_info_box(full_wiki_path))
        
    except Exception: #3 games won't open so I decided to pass them.
        pass

In [468]:
#Cheking the length of the list I created and saw that it has the same length
#without the games number I couldn't iterate.
len(game_info_list)

236

In [469]:
#Looked at the first 10 games to see that everything went well.
game_info_list[10:]

[{'Title': 'Donkey Kong',
  'Developer(s)': ['Nintendo R&D1[a]', 'Ikegami Tsushinki[1]'],
  'Publisher(s)': 'Nintendo',
  'Director(s)': 'Shigeru Miyamoto',
  'Producer(s)': 'Gunpei Yokoi',
  'Designer(s)': ['Shigeru Miyamoto', 'Gunpei Yokoi'],
  'Artist(s)': 'Shigeru Miyamoto',
  'Composer(s)': 'Yukio Kaneoka',
  'Series': ['Donkey Kong', 'Mario'],
  'Platform(s)': ['Arcade',
   'Ports Game & Watch Atari 2600 Intellivision ColecoVision Coleco Mini-arcade Atari 8-bit Famicom/NES TI-99/4a IBM PC Commodore 64 VIC-20 MSX ZX Spectrum Amstrad CPC Atari 7800 Apple II Nintendo e-Reader Game Boy Advance',
   'Game & Watch',
   'Atari 2600',
   'Intellivision',
   'ColecoVision',
   'Coleco Mini-arcade',
   'Atari 8-bit',
   'Famicom/NES',
   'TI-99/4a',
   'IBM PC',
   'Commodore 64',
   'VIC-20',
   'MSX',
   'ZX Spectrum',
   'Amstrad CPC',
   'Atari 7800',
   'Apple II',
   'Nintendo e-Reader',
   'Game Boy Advance'],
  'Release': ['JP: July 9, 1981[2]NA: July 31, 1981EU: November 1981[3]',

In [429]:
#Created a function to save my list as a json file, since it's easier to save dictionaries
#in this file type.
def save(title, data):
    with open(title, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        
save("Video Games Data.json", game_info_list)

In [430]:
#Created a function to load my json file.
def load(title):
    with open(title, "r", encoding="utf8") as f:
        return json.load(f)
    
game_info_list = load("Video Games Data.json")

# Clean The Data:

### Remove references box from the data and the keys.
### remove whitespaces from the data.
### Remove (s) from the dictionary keys, special sign such as "./!" etc and strip whitespaces as well.
### Fix release dates.

In [624]:
#Clean up references (remove [1] etc.).


AttributeError: 'dict' object has no attribute 'replace'